# **Visualization for HuBMAP + HPA - Hacking the Human Body**
##### **Segment multi-organ functional tissue units**

### **Importing necessary modules & packages**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tqdm.notebook import tqdm
from multiprocessing import cpu_count

import imageio, tifffile
import cv2, math, sys
import glob, os, joblib

### **Loading Training DataFrame**

In [ ]:
train = pd.read_csv('../input/hubmap-organ-segmentation/train.csv')
display(train.head())
display(train.info())

In [ ]:
MAX_WIDTH = train['img_width'].max()
MAX_HEIGHT = train['img_height'].max()
N_CHANNELS = 3
IMG_SIZE = 640
PATCH_SIZE = 640
N_PATCHES_PER_IMAGE = (IMG_SIZE // PATCH_SIZE) ** 2
N_SAMPLES = len(train)
N_PATCHES = N_SAMPLES * N_PATCHES_PER_IMAGE

print(f'N_SAMPLES: {N_SAMPLES}, N_PATCHES: {N_PATCHES}, MAX_WIDTH: {MAX_WIDTH}, MAX_HEIGHT: {MAX_HEIGHT}')
print(f'IMG_SIZE: {IMG_SIZE}, PATCH_SIZE: {PATCH_SIZE}, N_PATCHES_PER_IMAGE: {N_PATCHES_PER_IMAGE}')

### **Explorative Data Analysis**

In [ ]:
train[['img_height', 'img_width']].value_counts().sort_values(ascending=False)

In [ ]:
display(train['data_source'].value_counts().to_frame())  # distribution of data source
display(train['pixel_size'].value_counts().to_frame())  # distribution of pixel size
display(train['tissue_thickness'].value_counts().to_frame())  # tissue thickness

In [ ]:
# age distribution bar graph
plt.figure(figsize=(8,5))
train['age'].plot(kind='hist')
plt.title('Age Distribution', size=24)
plt.xlim(0, 100)
plt.show()

In [ ]:
# organ distribution pi-chart
ax = plt.figure(figsize=(8, 8), facecolor='white')
train['organ'].value_counts().plot(kind='pie', autopct='%1.1f%%', title='Organ Distribution')
plt.show()

In [ ]:
# pi-chart distribution based on gender
ax = plt.figure(figsize=(8, 8), facecolor='white')
train['sex'].value_counts().plot(kind='pie', autopct='%1.1f%%', title='Sex Distribution')
plt.show()

### **Plotting Samples**

In [ ]:
def resize_tensor(tensor):
    return cv2.resize(tensor, [IMG_SIZE, IMG_SIZE], interpolation=cv2.INTER_CUBIC).astype(np.uint8)

In [ ]:
def get_mask(image_id):
    row = train.loc[train['id'] == image_id].squeeze()
    h, w = row[['img_height', 'img_width']]
    mask = np.zeros(shape=[h * w], dtype=np.uint8)
    s = row['rle'].split()
    starts, lengths = [ np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2]) ]
    starts -= 1
    ends = starts + lengths
    for lo, hi in zip(starts, ends):
        mask[lo : hi] = 1
        
    mask = mask.reshape([h, w]).T
    mask = resize_tensor(mask)
    mask = np.expand_dims(mask, axis=2)
    return mask

In [ ]:
def get_image(image_id, negative=False):
    image = tifffile.imread(f'/kaggle/input/hubmap-organ-segmentation/train_images/{image_id}.tiff')
    if len(image.shape) == 5:
        image = image.squeeze().transpose(1, 2, 0)
        
    # Reverse pixels to make tissue colored and background black
    if negative:
        image = image - image.min()
        image = image / (image.max() - image.min())
        image = image * 255
        image = 255 - image.astype(np.uint8)
        
    image = resize_tensor(image)
    return image

image = get_image(train.loc[0, 'id'], True)

In [ ]:
def show_image_and_masks(rows=4, cols=4):
    # Figure
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols*8, rows*6))
    # Unique Image Ids
    image_ids = train['id'].unique()
    
    for r in range(rows):
        image_id = image_ids[r]
        df_row = train.loc[train['id'] == image_id].head(1).squeeze()
        # Rad Image
        image = get_image(image_id)
        image_negative = get_image(image_id, negative=True)
        
        # Image Original
        axes[r, 0].imshow(image)
        axes[r, 0].set_title(f'Image {image_id} Raw', size=16)
        axes[r, 0].axis(False)
        
        # Image Negative Original
        axes[r, 1].imshow(image_negative)
        axes[r, 1].set_title(f'Image {image_id} Negative min: {image_negative.min()} max: {image_negative.max()}', size=16)
        axes[r, 1].axis(False)
        
        # Mask
        mask = get_mask(image_id)
        axes[r, 2].imshow(mask)
        axes[r, 2].set_title('Mask', size=16)
        axes[r, 2].axis(False)
        
        # Image with Mask
        axes[r, 3].imshow(image)
        axes[r, 3].imshow((mask * np.array([255, 0, 0])), alpha=0.50)
        axes[r, 3].set_title('Image and Mask', size=16)
        axes[r, 3].axis(False)
            
    # Adjust Vertical Space Between Subplots
    fig.subplots_adjust(wspace=0.10)

In [ ]:
show_image_and_masks(rows=8)

### **Creating Train Arrays**

In [ ]:
def extract_patches(image):
    _, _, c = image.shape
    image = tf.expand_dims(image, 0)
    image_patches = tf.image.extract_patches(image, [1,PATCH_SIZE,PATCH_SIZE,1], [1, PATCH_SIZE, PATCH_SIZE, 1], [1, 1, 1, 1], padding='SAME')
    image_patches = tf.reshape(image_patches, [N_PATCHES_PER_IMAGE, PATCH_SIZE, PATCH_SIZE, c])
    image_patches = image_patches.numpy()

    return image_patches

In [ ]:
def get_image_mask(image_id):    
    image = get_image(image_id, True)
    image_patches = extract_patches(image)
    
    mask = get_mask(image_id)
    mask_patches = extract_patches(mask)
    
    organ = str.encode(train.loc[train['id'] == image_id, 'organ'].squeeze())
    return image_patches, mask_patches, organ

In [ ]:
N_CHUNKS = len(train)
CHUNKS = np.array_split(train['id'].values, N_CHUNKS)

print(f'N_CHUNKS: {N_CHUNKS}, CHUNK_SIZE: {len(CHUNKS[0])}')

In [ ]:
def to_tf_records(chunks):
    N_SAMPLES_PER_TFRECORD = []
    ORGAN_PER_TFRECORD = []
    for chunk_idx, image_id in enumerate(tqdm(chunks)):
        # Get Image and Mask Patches
        image_patches, mask_patches, organ = get_image_mask(image_id.squeeze())
        tfrecord_name = f'batch_{chunk_idx}.tfrecords'
        
        # Create the actual TFRecords
        with tf.io.TFRecordWriter(tfrecord_name) as file_writer:
            sample_count = 0
            for image, mask in zip(image_patches, mask_patches):
                sample_count += 1

                image_serialized = tf.io.serialize_tensor(image).numpy()
                mask_serialized = tf.io.serialize_tensor(mask).numpy()

                record_bytes = tf.train.Example(features=tf.train.Features(feature={
                    # Image
                    'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_serialized])),
                    # Mask
                    'mask': tf.train.Feature(bytes_list=tf.train.BytesList(value=[mask_serialized])),
                    
                    # Organ
                    'organ': tf.train.Feature(bytes_list=tf.train.BytesList(value=[organ])),
                })).SerializeToString()
                file_writer.write(record_bytes)
                    
            # Add Sample Count
            N_SAMPLES_PER_TFRECORD.append(sample_count)
            # Add organ
            ORGAN_PER_TFRECORD.append(organ)
            
    # Save Number of Samples per TFRecord to determine step count during training
    np.save('N_SAMPLES_PER_TFRECORD.npy', np.array(N_SAMPLES_PER_TFRECORD, dtype=np.int16))
    # Save organ per TFRecord for stratifying kfolds
    np.save('ORGAN_PER_TFRECORD.npy', np.array(ORGAN_PER_TFRECORD, dtype=str))

# Create TFRecords
to_tf_records(CHUNKS)

### **Checking TFRecord**

In [ ]:
def decode_tfrecord(record_bytes):
    features = tf.io.parse_single_example(record_bytes, {
        'image': tf.io.FixedLenFeature([], tf.string),
        'mask': tf.io.FixedLenFeature([], tf.string),
        'organ': tf.io.FixedLenFeature([], tf.string),
    })
    
    image = tf.io.parse_tensor(features['image'], out_type=tf.uint8)
    image = tf.reshape(image, [PATCH_SIZE, PATCH_SIZE, N_CHANNELS])
    
    mask = tf.io.parse_tensor(features['mask'], out_type=tf.uint8)
    mask = tf.reshape(mask, [PATCH_SIZE, PATCH_SIZE, 1])

    organ = features['organ']
    
    # Explicit reshape needed for TPU, tell cimpiler dimensions of image
    image = tf.reshape(image, [PATCH_SIZE, PATCH_SIZE, N_CHANNELS])
    # Explicit reshape needed for TPU, tell cimpiler dimensions of image
    mask = tf.reshape(mask, [PATCH_SIZE, PATCH_SIZE, 1])
    
    return image, mask, organ

In [ ]:
def get_train_dataset(bs):
    FNAMES_TRAIN_TFRECORDS = tf.io.gfile.glob('./*.tfrecords')
    train_dataset = tf.data.TFRecordDataset(FNAMES_TRAIN_TFRECORDS, num_parallel_reads=1)
    train_dataset = train_dataset.map(decode_tfrecord, num_parallel_calls=cpu_count())
    train_dataset = train_dataset.batch(bs)
    
    return train_dataset

### **Plotting Patches**

In [ ]:
def show_batch(dataset, rows=10, cols=2):
    images, masks, organs = next(iter(dataset))
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols*6, rows*6))
    for r in range(rows):
        axes[r, 0].imshow(images[r])
        organ = organs[r].numpy().decode("UTF-8")
        axes[r, 0].set_title(f'Organ: {organ}', size=16)
        axes[r, 1].imshow(masks[r])
        axes[r, 1].set_title('Mask', size=16)

In [ ]:
train_dataset = get_train_dataset(10)
show_batch(train_dataset)

### **Reconstruct Image/Mask from Patches**

In [ ]:
def show_image_mask_from_patches(dataset, n_samples=10):
    s = int(N_PATCHES_PER_IMAGE ** 0.50)
    counter = 0
    dataset_iter = iter(dataset)
    
    for _ in tqdm(range(n_samples)):
        # Main plot
        fig = plt.figure(figsize=(30, 10))
        subfigs = fig.subfigures(1, 3)
        plt.suptitle('Patched Image/Mask/Combined', fontsize=48, y=1.1)
        images, masks, organs = next(dataset_iter)
        o = organs[0].numpy().decode()
        
        # Make subplots
        ax_images = subfigs[0].subplots(s, s)
        subfigs[0].suptitle(f'Image ({o})', size=32)
        ax_masks = subfigs[1].subplots(s, s)
        subfigs[1].suptitle('Mask', size=32)
        ax_combined = subfigs[2].subplots(s, s)
        subfigs[2].suptitle('Combined', size=32)
        
        count = 0
        for r in range(s):
            for c in range(s):
                idx = r * s + c
                
                if s > 1:
                    # Image
                    ax_images[r,c].imshow(images[idx].numpy())
                    ax_images[r,c].set_title(f'σ{images[idx].numpy().std():.2f}', c='red')
                    ax_images[r,c].axis(False)
                    # Mask
                    ax_masks[r,c].imshow(masks[idx].numpy() * [255,255,0])
                    ax_masks[r,c].axis(False)
                    # Combined
                    ax_combined[r,c].imshow(images[idx].numpy())
                    ax_combined[r,c].imshow((masks[idx] * np.array([255, 0, 0])), alpha=0.50)
                    ax_combined[r,c].axis(False)
                else:
                    # Image
                    ax_images.imshow(images[idx].numpy())
                    ax_images.set_title(f'σ{images[idx].numpy().std():.2f}', c='red')
                    ax_images.axis(False)
                    # Mask
                    ax_masks.imshow(masks[idx].numpy() * [255,255,0])
                    ax_masks.axis(False)
                    # Combined
                    ax_combined.imshow(images[idx].numpy())
                    ax_combined.imshow((masks[idx] * np.array([255, 0, 0])), alpha=0.50)
                    ax_combined.axis(False)
                
                count += 1
                
        # Show plot
        fig.subplots_adjust(wspace=0.05)
        plt.show()

In [ ]:
train_dataset = get_train_dataset(N_PATCHES_PER_IMAGE)
show_image_mask_from_patches(train_dataset)

### **Mean/Standard Deviation Computation**

In [ ]:
MEAN = np.zeros(3, dtype=np.float32)
STD = np.zeros(3, dtype=np.float32)

for image, _, _ in tqdm(get_train_dataset(1), total=N_PATCHES):
    # Update Mean and STD
    image_np = image.numpy().squeeze()
    MEAN += (image_np / 255).mean(axis=(0,1)) / N_PATCHES
    STD += (image_np /  255).std(axis=(0,1)) / N_PATCHES

display(pd.Series(MEAN).to_frame('Mean'))
display(pd.Series(STD).to_frame('Standard Deviation'))

In [ ]:
np.save('MEAN.npy', MEAN)
np.save('STD.npy', STD)